In [2]:
# QuantBook Analysis Tool 
# For more information see [https://www.quantconnect.com/docs/v2/our-platform/research/getting-started]
qb = QuantBook()

## Pull FX and CFD data

In [1]:
forex_pairs = ['CADUSD', 'CHFUSD', 'EURUSD', 'GBPUSD', 'JPYUSD', 'NZDUSD', 'AUDUSD']
commodities = ['XAUUSD', 'XAGUSD', 'WTICOUSD', 'BCOUSD', 'CORNUSD', 'SOYBNUSD', 'XPDUSD', 'XPTUSD', 'NATGASUSD', 'SUGARUSD']


In [3]:
from IPython.display import clear_output

def pull_qc_data(forex_pairs, commodities, n, resolution = Resolution.Hour, market = Market.Oanda):
    history = []

    # Fetch Forex data
    for forex_pair in forex_pairs:
        forex_ticker = qb.AddForex(forex_pair, resolution, market).Symbol
        history.append(qb.History(forex_ticker, n, resolution).drop_duplicates())
        print(f"Pulling {forex_ticker}: done")

    # Fetch Commodity data
    for commodity in commodities:
        commodity_symbol = qb.AddCfd(commodity, resolution, market).Symbol
        history.append(qb.History(commodity_symbol, n, resolution).drop_duplicates())
        print(f"Pulling {commodity_symbol}: done")

    return history
    

In [4]:
history = pull_qc_data(forex_pairs, commodities, 24*10000, Resolution.Hour)


## Preprocessing

In [5]:

from statsmodels.tsa.stattools import coint
import pandas as pd
from typing import List


def preprocess_qc_data(history, ffill=True):

    history_dfs = pd.concat(history).reset_index()
    history_dfs['time'] = pd.to_datetime(history_dfs['time'])
    # Filter the pivoted DataFrame
    if ffill:   
        history_filt = history_dfs.pivot(index='time', columns='symbol', values='close').fillna(method='ffill')
    else:   
        history_filt = history_dfs.pivot(index='time', columns='symbol', values='close')

    return history_filt

def drop_most_nan_pairs(df, n=1):
    # Count NaN values for each column and sort
    nan_counts = df.isna().sum().sort_values(ascending=False)
    
    # Get the 10 forex pairs with the most NaN values
    most_nan_pairs = nan_counts.nlargest(n).index
    
    # Drop these pairs from the DataFrame
    df = df.drop(columns=most_nan_pairs)
    
    return df.dropna()

def calculate_spread(df_train, df_val):
    lr = LinearRegression()
    spreads_train = pd.DataFrame(index=df_train.index)
    spreads_val = pd.DataFrame(index=df_val.index)
    
    for symbol in df_train.columns:
        if symbol == 'EURUSD':
            continue
        
        # reshape for sklearn
        X_train = df_train[symbol].values.reshape(-1,1)
        y_train = df_train['EURUSD'].values.reshape(-1,1)

        # fit the model
        lr.fit(X_train, y_train)
        
        # use the coefficient to calculate spread
        spreads_train[symbol] = df_train['EURUSD'] - lr.coef_[0] * df_train[symbol]
        spreads_val[symbol] = df_val['EURUSD'] - lr.coef_[0] * df_val[symbol]

    return spreads_train, spreads_val


def split_data(df: pd.DataFrame) -> (List[pd.DataFrame], List[pd.DataFrame]):
    
    # Convert the index to DateTimeIndex if it's not already
    if not isinstance(df.index, pd.DatetimeIndex):
        df.index = pd.to_datetime(df.index)

    # Sort the DataFrame by date
    df.sort_index(inplace=True)

    # Determine the number of months in the dataset
    num_months = (df.index.max() - df.index.min()).days // 30
    
    train_data = []
    val_data = []

    # Split data into 4-month blocks with 1-month overlap and further into training and validation sets
    for i in range(0, num_months, 1):  # change here
        # Determine the cutoff dates for each period
        start_date = df.index.min() + pd.DateOffset(months=i)
        middle_date = start_date + pd.DateOffset(months=3)
        end_date = start_date + pd.DateOffset(months=4)

        # Split the data and append to the respective lists
        train_data.append(df[(df.index >= start_date) & (df.index < middle_date)])
        val_data.append(df[(df.index >= middle_date) & (df.index < end_date)])

    return train_data, val_data



In [6]:
history_fx_filt = preprocess_qc_data(history)
history_fx_filt = drop_most_nan_pairs(history_fx_filt)

In [7]:
train_history, validate_history = split_data(history_fx_filt.iloc[50000:])

In [8]:
def check_unique_dates(val_data):
    all_dates = []

    # Iterate over all dataframes in val_data
    for df in val_data:
        all_dates.extend(df.index.tolist())
    
    # Check if the number of unique dates is equal to the total number of dates
    return len(all_dates) == len(set(all_dates))

print(f"Sanity check for timestamps: {check_unique_dates(validate_history)}")


## Get Cointegrated pairs

### Using Engle-Granger test and Kendall's tau to measure rank correlation

In [9]:
def run_engle_granger(forex_pairs, df):

    cointegrated_pairs = []

    for i in range(len(forex_pairs)):
        for j in range(i+1, len(forex_pairs)):

            forex_pair1 = forex_pairs[i]
            forex_pair2 = forex_pairs[j]

            eg_test_result = coint(df[forex_pair1], df[forex_pair2], trend='c', method='aeg', autolag='AIC')
            eg_output = pd.Series(eg_test_result[0:3], index=['Test Statistic','p-value','Critical Values 1%,5%,10%'])

            print(f'Results of Engle-Granger Test between {forex_pair1} and {forex_pair2}:')
            print(eg_output)

            if eg_test_result[1] < 0.1: # pairs with a p-value less than 0.01 are considered cointegrated
                cointegrated_pairs.append((forex_pair1, forex_pair2))


    return cointegrated_pairs

# Function to compute Kendall's Tau
def kendall_tau(x, y):
    return kendalltau(x, y)[0]  # we only want the correlation coefficient

def find_pair_with_highest_tau(df, pairs):
    """
    Finds the pair of columns in the DataFrame with the highest Kendall's Tau.
    Args:
        df (pandas.DataFrame): The DataFrame.
        pairs (list): The list of pairs to consider.

    Returns:
        tuple: The pair with the highest Kendall's Tau.
    """
    max_tau = -1  # initial value (minimum possible)
    best_pair = None

    for pair in pairs:
        tau = kendall_tau(df[pair[0]], df[pair[1]])
        if tau > max_tau:
            max_tau = tau
            best_pair = pair

    return best_pair

def find_pair_with_highest_corr(df, pairs):
    """
    Finds the pair of columns in the DataFrame with the highest Kendall's Tau.
    Args:
        df (pandas.DataFrame): The DataFrame.
        pairs (list): The list of pairs to consider.

    Returns:
        tuple: The pair with the highest Kendall's Tau.
    """
    max_tau = -1  # initial value (minimum possible)
    best_pair = None

    for pair in pairs:
        tau = df[pair[0]].corr(df[pair[1]])
        if tau > max_tau:
            max_tau = tau
            best_pair = pair

    return best_pair


### Calculate the hedge ratio between EURUSD and the other pairs 
(Old approach)

In [10]:
from sklearn.linear_model import LinearRegression
from scipy import stats
import numpy as np

In [11]:
from scipy.stats import kendalltau

def calculate_spread(df_train, df_val):
    lr = LinearRegression()
    spreads_train = pd.DataFrame(index=df_train.index)
    spreads_val = pd.DataFrame(index=df_val.index)
    
    for symbol in df_train.columns:
        if symbol == 'EURUSD':
            continue
        
        # reshape for sklearn
        X_train = df_train[symbol].values.reshape(-1,1)
        y_train = df_train['EURUSD'].values.reshape(-1,1)

        # fit
        lr.fit(X_train, y_train)
        
        # use the coefficient to calculate spread
        spreads_train[symbol] = df_train['EURUSD'] - lr.coef_[0] * df_train[symbol]
        spreads_val[symbol] = df_val['EURUSD'] - lr.coef_[0] * df_val[symbol]

    return spreads_train, spreads_val
    
def get_ecdfs(data_1, data_2):

    def get_ecdf(data, t):
        n = len(data)
        return sum([int(x < t) for x in data])/n

    data_1_trsf = list(map(lambda x: get_ecdf(data_1, x), data_1))
    data_2_trsf = list(map(lambda x: get_ecdf(data_2, x), data_2))

    return data_1_trsf, data_2_trsf



### Test on the first 4-month block 

In [13]:
# a, b = calculate_spread(train_history[0], validate_history[0])
a, b = train_history[0], validate_history[0]

In [23]:
coint_spreads = run_engle_granger(a.columns, a)

In [27]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kendalltau

In [25]:
best_spread = find_pair_with_highest_tau(a, coint_spreads)
copula_data = pd.concat([a[best_spread[0]], a[best_spread[1]]], axis=1)

In [31]:
copula_data = get_ecdfs(a[best_spread[0]], a[best_spread[1]])

In [15]:
import numpy as np
from copulas.multivariate import GaussianMultivariate
from copulas.bivariate import Gumbel

class GaussianMultivariateCustom(Gumbel):
    
    def partial_derivative_u(self, u, v):
        r"""Compute first partial derivative of cumulative distribution.

        The partial derivative of the copula(CDF) is the conditional CDF.

        .. math:: F(v|u) = \frac{\partial C(u,v)}{\partial u} =
            C(u,v)\frac{((-\ln u)^{\theta} + (-\ln v)^{\theta})^{\frac{1}{\theta} - 1}}
            {\theta(- \ln u)^{1 -\theta}}

        Args:
            X (np.ndarray)
            y (float)

        Returns:
            numpy.ndarray

        """
        X = np.column_stack((u, v))

        return self.partial_derivative(X)

    def partial_derivative_v(self, u, v):
            r"""Compute second partial derivative of cumulative distribution.

            The partial derivative of the copula(CDF) is the conditional CDF.

            .. math:: F(v|u) = \frac{\partial C(u,v)}{\partial u} =
                C(u,v)\frac{((-\ln u)^{\theta} + (-\ln v)^{\theta})^{\frac{1}{\theta} - 1}}
                {\theta(- \ln u)^{1 -\theta}}

            Args:
                X (np.ndarray)
                y (float)

            Returns:
                numpy.ndarray

            """
            X = np.column_stack((v, u))

            return self.partial_derivative(X)

In [29]:
copula = GaussianMultivariateCustom()
copula.fit(pd.DataFrame(copula_data).T.to_numpy())

In [51]:
cond_prob_u = copula.partial_derivative_u
cond_prob_v = copula.partial_derivative_v

In [52]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Define the resolution of the grid (100x100 grid here)
resolution = 100

# Generate the grid over the range [0,1]²
x = np.linspace(0, 1, resolution)
y = np.linspace(0, 1, resolution)
xx, yy = np.meshgrid(x, y)


# Compute the value of the function at each point in the grid
z_u = np.empty_like(xx)
z_v = np.empty_like(xx)

for i in range(resolution):
    for j in range(resolution):
        z_u[i, j] = cond_prob_u(xx[i, j], yy[i, j])
        z_v[i, j] = cond_prob_v(xx[i, j], yy[i, j])


In [54]:
sns.heatmap(pd.DataFrame(z_v).iloc[:, ::-1], cmap="YlOrBr")


In [55]:
plt.scatter(a[best_spread[0]], a[best_spread[1]], s=1)

In [44]:
plt.scatter(copula_data[0], copula_data[1], s=1)

In [47]:
fig, ax = plt.subplots()
fig.set_size_inches(10, 7)
sns.heatmap(history_fx_filt.corr().apply(lambda x: round(x, 4)), annot=True, cmap="YlOrBr")

In [357]:
from copulas.bivariate.gumbel import Gumbel
from copulas.bivariate.utils import split_matrix

In [360]:
copula = Gumbel()
copula.fit(pd.DataFrame(copula_data).T.to_numpy())

In [373]:
copula.partial_derivative_scalar(0.2, 0.4)

In [359]:
split_matrix(pd.DataFrame(copula_data).T.to_numpy())

## Put the copula's partial derivatives in QC ObjectStore 

In [12]:
from copulas.bivariate import Gumbel
from statsmodels.tsa.stattools import coint
from statsmodels.distributions.empirical_distribution import ECDF
from IPython.display import clear_output


def find_and_fit_best_spreads(train_histories, val_histories):

    copula_series = {}
    ecdfs_series = {}  # To store the ecdfs
    best_spread_series = {}  # To store the best spreads
    total = len(train_histories)

    for i, (train_df, val_df) in enumerate(zip(train_histories, val_histories)):
        
        clear_output(wait=True)
        percentage_done = (i + 1) / total * 100
        print(f"Progress: {percentage_done:.2f}%")

        if len(val_df) > 0:
            # Find cointegrated pairs
            av_pairs = list(train_df.columns)
            coint_pairs = run_engle_granger(av_pairs, train_df)

            # Find pair with highest tau
            best_spread = find_pair_with_highest_tau(train_df, coint_pairs)
            
            # Get empirical cumulative distribution functions of the spread
            copula_data = get_ecdfs(train_df[best_spread[0]], train_df[best_spread[1]])
            
            try:
                # Fit the copula
                copula = Gumbel()
                copula.fit(np.column_stack(copula_data))

                # Store the fitted copula, the ecdfs and the best spread
                copula_series[val_df.index[0]] = copula
                ecdfs_series[val_df.index[0]] = (ECDF(train_df[best_spread[0]]), ECDF(train_df[best_spread[1]]))
                best_spread_series[val_df.index[0]] = best_spread

            except ValueError:

                copula_series[val_df.index[0]] = np.nan
                ecdfs_series[val_df.index[0]] = np.nan
                best_spread_series[val_df.index[0]] = np.nan
    
    return pd.Series(copula_series), pd.Series(ecdfs_series), pd.Series(best_spread_series)


In [13]:
copula_series, ecdfs_series, best_spread_series = find_and_fit_best_spreads(train_history, validate_history)

In [15]:
copula_series.iloc[0].partial_derivative

In [14]:
import pickle

# Convert pandas Series to string, then encode to bytes for storage
copula_series_bytes = pickle.dumps(copula_series)
ecdfs_series_bytes = pickle.dumps(ecdfs_series)
best_spread_series_bytes = pickle.dumps(best_spread_series)

# Save data to Object Store
qb.ObjectStore.SaveBytes("copula_series_key", copula_series_bytes)
qb.ObjectStore.SaveBytes("ecdfs_series_key", ecdfs_series_bytes)
qb.ObjectStore.SaveBytes("best_spread_series_key", best_spread_series_bytes)


In [28]:
for kvp in qb.ObjectStore:
    key = kvp.Key
    value = kvp.Value
    print(f"{key}: {value}")

#### Multiprocessing (Incomplete)

Can only be run on docker using Lean CLI

In [ ]:
from multiprocessing import Pool

def process_data(args):

    i, train_df, val_df = args

    if len(val_df) > 0:

        # Find cointegrated pairs 
        av_pairs = list(train_df.columns)
        coint_pairs = run_engle_granger(av_pairs, train_df)

        # Find pair with highest tau
        best_spread = find_pair_with_highest_tau(train_df, coint_pairs)
        
        # Get empirical cumulative distribution functions of the spread
        copula_data = get_ecdfs(train_df[best_spread[0]], train_df[best_spread[1]])
        
        try:
            # Fit the copula
            copula = GaussianMultivariateCustom()
            copula.fit(np.column_stack(copula_data))

            # Store the fitted copula, the ecdfs and the best spread
            copula_series[val_df.index[0]] = copula
            ecdfs_series[val_df.index[0]] = (ECDF(train_df[best_spread[0]]), ECDF(train_df[best_spread[1]]))
            best_spread_series[val_df.index[0]] = best_spread

        except ValueError:

            copula_series[val_df.index[0]] = np.nan
            ecdfs_series[val_df.index[0]] = np.nan
            best_spread_series[val_df.index[0]] = np.nan    
            
        return (val_df.index[0], copula, ecdf, best_spread)

def find_and_fit_best_spreads_parallel(train_histories, val_histories, num_processes=4):
    with Pool(num_processes) as p:
        results = p.map(process_data, enumerate(zip(train_histories, val_histories)))

    copula_series = {k: v for k, v, _, _ in results}
    ecdf_series = {k: v for k, _, v, _ in results}
    best_spread_series = {k: v for k, _, _, v in results}

    return pd.Series(copula_series), pd.Series(ecdf_series), pd.Series(best_spread_series)


### Reading data from QC ObjectStore 
To build a sanity check function used in the backtesting framework

In [29]:
import pickle 
# from gumbel_copula import GaussianMultivariateCustom

def load_copulas(qb):
    # Import the series from the Object Store
    copula_series_object_ = qb.ObjectStore.ReadBytes("copula_series_key")
    ecdf_series_object_ = qb.ObjectStore.ReadBytes("ecdfs_series_key")
    best_spread_series_object_ = qb.ObjectStore.ReadBytes("best_spread_series_key")

    copula_series_ = pickle.loads(copula_series_object_)
    ecdf_series_ =  pickle.loads(ecdf_series_object_)
    best_spread_series_ = pickle.loads(best_spread_series_object_)

    return copula_series_, ecdf_series_, best_spread_series_

# copula_series.index = pd.to_datetime(copula_series.index)
# ecdf_series.index = pd.to_datetime(ecdf_series.index)
# best_spread_series.index = pd.to_datetime(best_spread_series.index)

In [30]:
tup = load_copulas(qb)

In [31]:
tup